In [29]:
import re

def strip_code_block(content: str) -> str:
    """
    如果字符串以 ``` 或 ```json 开头，并以 ``` 结尾，
    则去掉这些 Markdown 代码块标记，只保留中间的 JSON。
    """
    content = content.strip()
    content = re.sub(r"^```(json)?", "", content, flags=re.MULTILINE)
    content = re.sub(r"```$", "", content, flags=re.MULTILINE)
    return content.strip()

# ========== 1. 基础配置 ==========

API_KEY = "sk-dd280b67097548a8ab1b1ccd9b767569"  # 请替换为你的真实 API Key
INPUT_EXCEL = "TEXT.xlsx"       # 输入的 Excel 文件
OUTPUT_EXCEL = "output_file4.xlsx"   # 输出结果的 Excel 文件

BASE_URL = "https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions"

VALID_CATEGORIES = {
    "奥运", "国内政治", "国际政治", "军事和防御", "国内秩序", "经济", "劳动关系", "商务活动", "交通",
    "卫生福利", "人口", "教育", "传播", "住房", "环境", "能源", "科技", "社会关系", "灾害", "体育",
    "文化活动", "时尚", "庆典", "人类爱好", "其他"
}
VALID_ATTITUDES = {"positive", "negative", "hostile", "pleasure", "pain"}

# 构建基础的 Payload 配置（注意如果有嵌套字典，copy() 只是浅拷贝）
BASE_PAYLOAD = {
    "model": "qwen-max-latest",  # 请根据需要修改模型名称
    "stream": False,
    "temperature": 0.7,
    "top_p": 0.4,
    "presence_penalty": 0,
    "response_format": {"type": "text"},
    "tools": [
        {
            "type": "function",
            "function": {
                "description": "<string>",
                "name": "<string>",
                "parameters": {},
                "strict": False
            }
        }
    ]
}

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

In [30]:
# ========== 2. 定义辅助函数 ==========
import time
import json
import requests
from IPython.display import display, update_display

# 全局的调试输出 display id
DEBUG_DISPLAY_ID = "debug_display"

# 创建调试信息输出区域（如果该区域还未创建）
display("", display_id=DEBUG_DISPLAY_ID)

def update_debug(msg: str):
    """
    更新调试信息显示，不清空其他输出（如 tqdm 进度条）
    """
    update_display(msg, display_id=DEBUG_DISPLAY_ID, clear=True)

def normalize_category(raw_category: str) -> str:
    """
    将模型输出的分类修正为预定义的值，不匹配时返回 '其他'
    """
    raw_category = raw_category.strip()
    for cat in VALID_CATEGORIES:
        if cat in raw_category:
            return cat
    return "其他"

def normalize_attitude(raw_attitude: str) -> str:
    """
    将模型输出的情感修正为预定义的值（仅允许返回 [positive, negative, hostile, pleasure, pain] 中的一个）。
    如果无法匹配，则返回默认值 "positive"（你也可以根据实际情况选择其他值，但必须在给定范围内）。
    """
    raw_attitude = raw_attitude.strip().lower()
    for att in VALID_ATTITUDES:
        if att in raw_attitude:
            return att
    return "positive"  # 默认返回 "positive"，确保返回值属于给定范围

def get_classification_and_sentiment(title: str, text: str) -> (str, str):
    """
    一次 API 请求，获取文章的分类和情感。
    内部增加了重试机制，确保在获得完整回复后才返回结果。
    """
    prompt = f"""
请阅读以下文章，并给出以下信息（请严格用JSON格式返回）：
1. classification：只能从下列类别中选择：{list(VALID_CATEGORIES)}；如果无法判断，请将 classification 设置为 "其他"。
2. sentiment：只能从下列五个值中选择：[positive, negative, hostile, pleasure, pain]；请务必选择其中一个最合适的值，不允许返回其他值。

请务必返回 JSON 格式（请注意不要输出任何“```”等多余字符），格式如下：
{{
  "classification": "xxx",
  "sentiment": "xxx"
}}

文章标题：{title}
文章内容：{text}
"""
    # 注意这里用浅拷贝即可，messages 字段会覆盖掉原有的同名字段
    payload = BASE_PAYLOAD.copy()
    payload["messages"] = [
        {
            "role": "user",
            "content": prompt
        }
    ]
    
    max_retries = 5
    retry_count = 0
    while retry_count < max_retries:
        try:
            response = requests.post(BASE_URL, json=payload, headers=HEADERS)
            if response.status_code == 429:
                update_debug(f"触发速率限制 (HTTP 429)，等待60秒后重试... 剩余重试次数: {max_retries - retry_count}")
                time.sleep(60)
                retry_count += 1
                continue
            elif response.status_code != 200:
                update_debug(f"服务器返回状态码 {response.status_code}，等待10秒后重试... 剩余重试次数: {max_retries - retry_count}")
                time.sleep(10)
                retry_count += 1
                continue

            # 成功获得回复后，更新调试输出区域显示返回信息
            debug_message = f"===== 调试信息：服务器返回 =====\n{response.text}\n============================="
            update_debug(debug_message)

            response_json = response.json()
            if "choices" not in response_json or len(response_json["choices"]) == 0:
                update_debug("未获得有效的 choices，返回默认值")
                return "其他", "positive"

            raw_content = response_json["choices"][0]["message"]["content"]
            raw_content = strip_code_block(raw_content)
            result = json.loads(raw_content)

            raw_cat = result.get("classification", "其他")
            raw_sen = result.get("sentiment", "positive")
            category = normalize_category(raw_cat)
            sentiment = normalize_attitude(raw_sen)
            return category, sentiment

        except Exception as e:
            update_debug(f"请求或解析时出现异常：{e}，等待10秒后重试... 剩余重试次数: {max_retries - retry_count}")
            time.sleep(10)
            retry_count += 1

    update_debug("达到最大重试次数，返回默认值")
    return "其他", "positive"


''

In [ ]:
# ========== 3. 主流程 ==========
import pandas as pd
import time
from tqdm.notebook import tqdm  # 使用 notebook 版 tqdm

# 建议在本 cell 的开始处创建调试输出区域（如果辅助函数所在 cell 未执行，也可确保该区域存在）
from IPython.display import display
display("", display_id=DEBUG_DISPLAY_ID)

df = pd.read_excel(INPUT_EXCEL)
results = []

# 使用 tqdm 的 notebook 版本来实现原地刷新进度条
with tqdm(total=df.shape[0], desc="Processing") as pbar:
    for idx, row in df.iterrows():
        art_id = row['id']
        title = row['title']
        text = row['text']

        category, sentiment = get_classification_and_sentiment(title, text)
        results.append({
            'id': art_id,
            'title': title,
            'text': text,
            'classification': category,
            'sentiment': sentiment
        })
        
        pbar.update(1)
        time.sleep(1)


KeyboardInterrupt: 

In [27]:
result_df = pd.DataFrame(results).sort_values(by='id')
result_df.to_excel(OUTPUT_EXCEL, index=False)
print(f"处理完成，结果已保存到：{OUTPUT_EXCEL}")

处理完成，结果已保存到：output_file3.xlsx
